### Simple Intorduction
- SqlAlchemy is a python based ORM(Object Relationship Mapper) Tool that makes us intract with the database using python instead of actual SQL queries
- It consists of **TWO main parts** 
    - SqlAlchemy Core (Low Level)
    - SqlAlchemy ORM (High Level)

# SqlAlchemy Core

### Engine and Connection Creation

In [3]:
from sqlalchemy import create_engine, text

In [4]:
DATABASE_URL = "sqlite:///CRUDOperations.db"
engine = create_engine(DATABASE_URL, echo=True) # We can also create connection pools to efficiently make multiply connections
conn = engine.connect()

### Basic table creation and deletion query

**note**: Here in DDL(CREATE, ALTER, DROP) commands there is no need to commit, and rollback will have no effect

In [5]:
conn.execute(text("CREATE TABLE IF NOT EXISTS person (id int, name str, age int)"))

2025-09-21 02:26:29,026 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,027 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS person (id int, name str, age int)
2025-09-21 02:26:29,027 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ()


In [6]:
conn.execute(text("DROP TABLE person"))

2025-09-21 02:26:29,038 INFO sqlalchemy.engine.Engine DROP TABLE person
2025-09-21 02:26:29,038 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()


### Working on actual SQLAlchemy Core

In [7]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData

In [8]:
engine = create_engine(DATABASE_URL, echo=True)
meta = MetaData() # MetaData is basically a registry / container object that holds information about your database schema.
conn = engine.connect()

#### Table Creation

In [9]:
person = Table(
    "person",
    meta,
    Column("id", Integer, primary_key=True),
    Column("name", String, nullable=False),
    Column("age", Integer, nullable=False)
)

meta.create_all(engine)

2025-09-21 02:26:29,084 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,086 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("person")
2025-09-21 02:26:29,087 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 02:26:29,089 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("person")
2025-09-21 02:26:29,089 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 02:26:29,091 INFO sqlalchemy.engine.Engine 
CREATE TABLE person (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	age INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2025-09-21 02:26:29,091 INFO sqlalchemy.engine.Engine [no key 0.00055s] ()
2025-09-21 02:26:29,094 INFO sqlalchemy.engine.Engine COMMIT


#### Record Insertion

In [10]:
insert_statement = person.insert().values(name="Mike", age=30)
result = conn.execute(insert_statement)
conn.commit()

2025-09-21 02:26:29,104 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,106 INFO sqlalchemy.engine.Engine INSERT INTO person (name, age) VALUES (?, ?)
2025-09-21 02:26:29,107 INFO sqlalchemy.engine.Engine [generated in 0.00291s] ('Mike', 30)
2025-09-21 02:26:29,108 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
from sqlalchemy import insert
insert_statement = insert(person).values(name="Tanvi", age=24)
result = conn.execute(insert_statement)
conn.commit()

2025-09-21 02:26:29,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,118 INFO sqlalchemy.engine.Engine INSERT INTO person (name, age) VALUES (?, ?)
2025-09-21 02:26:29,118 INFO sqlalchemy.engine.Engine [cached since 0.01496s ago] ('Tanvi', 24)
2025-09-21 02:26:29,120 INFO sqlalchemy.engine.Engine COMMIT


#### Record Selections

In [12]:
select_statement = person.select().where(person.c.age > 20)
result = conn.execute(select_statement)

for record in result:
    print(record)

2025-09-21 02:26:29,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,130 INFO sqlalchemy.engine.Engine SELECT person.id, person.name, person.age 
FROM person 
WHERE person.age > ?
2025-09-21 02:26:29,130 INFO sqlalchemy.engine.Engine [generated in 0.00126s] (20,)
(1, 'Mike', 30)
(2, 'Tanvi', 24)


#### Updating Records

In [13]:
update_statement = person.update().where(person.c.name == "Mike").values(age=50)
conn.execute(update_statement)
conn.commit()

result = conn.execute(person.select())
for record in result:
    print(record)

2025-09-21 02:26:29,142 INFO sqlalchemy.engine.Engine UPDATE person SET age=? WHERE person.name = ?
2025-09-21 02:26:29,142 INFO sqlalchemy.engine.Engine [generated in 0.00124s] (50, 'Mike')
2025-09-21 02:26:29,142 INFO sqlalchemy.engine.Engine COMMIT
2025-09-21 02:26:29,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,149 INFO sqlalchemy.engine.Engine SELECT person.id, person.name, person.age 
FROM person
2025-09-21 02:26:29,149 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ()
(1, 'Mike', 50)
(2, 'Tanvi', 24)


#### Record Deletion

In [14]:
delete_statement = person.delete().where(person.c.name == "Mike")
conn.execute(delete_statement)
conn.commit()

result = conn.execute(person.select())
for record in result:
    print(record)

2025-09-21 02:26:29,157 INFO sqlalchemy.engine.Engine DELETE FROM person WHERE person.name = ?
2025-09-21 02:26:29,157 INFO sqlalchemy.engine.Engine [generated in 0.00135s] ('Mike',)
2025-09-21 02:26:29,161 INFO sqlalchemy.engine.Engine COMMIT
2025-09-21 02:26:29,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,165 INFO sqlalchemy.engine.Engine SELECT person.id, person.name, person.age 
FROM person
2025-09-21 02:26:29,166 INFO sqlalchemy.engine.Engine [cached since 0.01813s ago] ()
(2, 'Tanvi', 24)


#### Create A Relation using ForeignKey

In [15]:
from sqlalchemy import ForeignKey, Float

things = Table(
    "things",
    meta,
    Column("id", Integer, primary_key=True),
    Column("description", String, nullable=False),
    Column("value", Float, nullable=False),
    Column("owner", Integer, ForeignKey("person.id"))
)

meta.create_all(bind=engine)

2025-09-21 02:26:29,177 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,178 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("person")
2025-09-21 02:26:29,179 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 02:26:29,180 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("things")
2025-09-21 02:26:29,181 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 02:26:29,182 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
insert_statement = person.insert().values([
    {"name":"Mike", "age":30},
    {"name":"Randy", "age":42},
    {"name":"Jhon", "age":28},
    {"name":"Steve", "age":32},
    {"name":"Ross", "age":24},
    {"name":"Rachel", "age":22},
])
conn.execute(insert_statement)
conn.commit()

2025-09-21 02:26:29,190 INFO sqlalchemy.engine.Engine INSERT INTO person (name, age) VALUES (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?)
2025-09-21 02:26:29,191 INFO sqlalchemy.engine.Engine [no key 0.00108s] ('Mike', 30, 'Randy', 42, 'Jhon', 28, 'Steve', 32, 'Ross', 24, 'Rachel', 22)
2025-09-21 02:26:29,192 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
insert_statement = things.insert().values([
    {"owner":2, "description":"Laptop", "value":800.50},
    {"owner":2, "description":"Mouse", "value":10.99},
    {"owner":2, "description":"KeyBoard", "value":20.99},
    {"owner":3, "description":"Speaker", "value":50.00},
    {"owner":4, "description":"Monitor", "value":40.30},
    {"owner":5, "description":"Mobile", "value":500.50},
])

conn.execute(insert_statement)
conn.commit()

2025-09-21 02:26:29,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,203 INFO sqlalchemy.engine.Engine INSERT INTO things (description, value, owner) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?)
2025-09-21 02:26:29,203 INFO sqlalchemy.engine.Engine [no key 0.00153s] ('Laptop', 800.5, 2, 'Mouse', 10.99, 2, 'KeyBoard', 20.99, 2, 'Speaker', 50.0, 3, 'Monitor', 40.3, 4, 'Mobile', 500.5, 5)
2025-09-21 02:26:29,206 INFO sqlalchemy.engine.Engine COMMIT


####  Join Statements

In [18]:
join_statement = person.join(things, person.c.id == things.c.owner)
select_statement = person.select().with_only_columns(person.c.name, things.c.description).select_from(join_statement)
result = conn.execute(select_statement)

for record in result:
    print(record)

2025-09-21 02:26:29,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,236 INFO sqlalchemy.engine.Engine SELECT person.name, things.description 
FROM person JOIN things ON person.id = things.owner
2025-09-21 02:26:29,237 INFO sqlalchemy.engine.Engine [generated in 0.00153s] ()
('Tanvi', 'Laptop')
('Tanvi', 'Mouse')
('Tanvi', 'KeyBoard')
('Mike', 'Speaker')
('Randy', 'Monitor')
('Jhon', 'Mobile')
('Tanvi', 'Laptop')
('Tanvi', 'Mouse')
('Tanvi', 'KeyBoard')
('Mike', 'Speaker')
('Randy', 'Monitor')
('Jhon', 'Mobile')


#### Group-BY Statements

In [19]:
from sqlalchemy import func
group_by_statement = things.select().with_only_columns(things.c.owner, func.sum(things.c.value)).group_by(things.c.owner)

result = conn.execute(group_by_statement)
for record in result:
    print(record)

2025-09-21 02:26:29,246 INFO sqlalchemy.engine.Engine SELECT things.owner, sum(things.value) AS sum_1 
FROM things GROUP BY things.owner
2025-09-21 02:26:29,247 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
(2, 864.46)
(3, 100)
(4, 80.6)
(5, 1001.0)


# SQLAlchemy ORM

In [20]:
from sqlalchemy import Column, String, Integer, ForeignKey, create_engine, func
from sqlalchemy.orm import sessionmaker, declarative_base, relationship

### Engine creation and connection establishment

In [21]:
DATABASE_URL = "sqlite:///CRUDOperations.db"
engine = create_engine(DATABASE_URL, echo=True)

### Table Creation

In [22]:
Base = declarative_base()

class User(Base):
    __tablename__ = "user"
    user_id: int = Column(Integer, primary_key=True)
    name: str = Column(String, nullable=False)
    email: str = Column(String, nullable=False)

    blogs = relationship("Blog", back_populates="author")

class Blog(Base):
    __tablename__ = "blog"
    blog_id: int = Column(Integer, primary_key=True)
    title: str = Column(String, nullable=False)
    description: str = Column(String, nullable=False)
    author_id: int = Column(Integer, ForeignKey("user.user_id"))

    author = relationship("User", back_populates="blogs")

In [23]:
Base.metadata.create_all(engine)

2025-09-21 02:26:29,413 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,414 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2025-09-21 02:26:29,414 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 02:26:29,415 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("blog")
2025-09-21 02:26:29,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 02:26:29,417 INFO sqlalchemy.engine.Engine COMMIT


### Session creation and accessing the data

In [24]:
Session = sessionmaker(bind=engine)
session = Session()

In [25]:
new_user = User(name="Rashwanth", email="rashwanth@gmail.com")
session.add(new_user)
session.commit()

2025-09-21 02:26:29,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,436 INFO sqlalchemy.engine.Engine INSERT INTO user (name, email) VALUES (?, ?)
2025-09-21 02:26:29,436 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('Rashwanth', 'rashwanth@gmail.com')
2025-09-21 02:26:29,439 INFO sqlalchemy.engine.Engine COMMIT


### Selecting data

In [26]:
kavin = session.query(User).filter(User.name=="Kavin").first()

2025-09-21 02:26:29,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,450 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2025-09-21 02:26:29,451 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('Kavin', 1, 0)


In [27]:
all_record = session.query(User).all()

2025-09-21 02:26:29,458 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user
2025-09-21 02:26:29,459 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()


In [28]:
new_blog = Blog(title="How to successfully lose in life!", description="Buy this one and dig a 6 foot grave and jump into it!", author_id=kavin.user_id)
session.add(new_blog)
session.commit()

2025-09-21 02:26:29,467 INFO sqlalchemy.engine.Engine INSERT INTO blog (title, description, author_id) VALUES (?, ?, ?)
2025-09-21 02:26:29,468 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('How to successfully lose in life!', 'Buy this one and dig a 6 foot grave and jump into it!', 1)
2025-09-21 02:26:29,469 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
session.query(Blog).first().author.name

2025-09-21 02:26:29,477 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,478 INFO sqlalchemy.engine.Engine SELECT blog.blog_id AS blog_blog_id, blog.title AS blog_title, blog.description AS blog_description, blog.author_id AS blog_author_id 
FROM blog
 LIMIT ? OFFSET ?
2025-09-21 02:26:29,479 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (1, 0)
2025-09-21 02:26:29,481 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.user_id = ?
2025-09-21 02:26:29,481 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (1,)


'Kavin'

### Deleting and Updating Record

In [30]:
session.query(User).filter(User.name=="Rashwanth").delete()
session.commit()

2025-09-21 02:26:29,490 INFO sqlalchemy.engine.Engine DELETE FROM user WHERE user.name = ?
2025-09-21 02:26:29,491 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ('Rashwanth',)
2025-09-21 02:26:29,493 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
session.query(User).filter(User.name=="Kavin").update({"email":"kavin@gmail.com"})
session.commit()

2025-09-21 02:26:29,501 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,503 INFO sqlalchemy.engine.Engine UPDATE user SET email=? WHERE user.name = ?
2025-09-21 02:26:29,504 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('kavin@gmail.com', 'Kavin')
2025-09-21 02:26:29,505 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
### Joints

In [33]:
session.query(User.name, func.count(Blog.blog_id).label("Total No Blogs")).join(User.blogs).group_by(User.user_id).all()

2025-09-21 02:26:29,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 02:26:29,520 INFO sqlalchemy.engine.Engine SELECT user.name AS user_name, count(blog.blog_id) AS "Total No Blogs" 
FROM user JOIN blog ON user.user_id = blog.author_id GROUP BY user.user_id
2025-09-21 02:26:29,520 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ()


[('Kavin', 4)]